In [10]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Name 1: Jorge Olmos
- Name 2: Bosang Yun
- Name 3: Yucheol Shin
- Name 4: Siu Wai Yeung

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [1]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset?
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

___
Enter your answer here:

1. There are 11314 instances of news articles
2. Each instance represents a words/phrases in a news article 
3. There are 20 classes, each class represent a newsgroup (category)
4. No, we don't know when this dataset was gathered and it was last modified in 2008. The data could contain old references, vocubulary, names, spelling, memes, etc...
5. The data is represented in sparse matrix 




___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [50]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean(a,c), 'bc:',euclidean(b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:', cosine(b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard(a>0,b>0), 'ac:', jaccard(a>0,c>0), 'bc:', jaccard(b>0,c>0))

print('\n\nThe most appropriate distance is...')
print('Cosine is the most appropriate distance and it does the best at classifying related documents. It helps overcome the flaw in \'count-the-common-words\' used in Euclidean distance')

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398234 bc: 0.9177794226661624
Cosine Distance
 ab: 0.6033714113755322 ac: 0.7070276149559529 bc: 0.4211595343347173
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.8821138211382114 ac: 0.8754716981132076 bc: 0.9087947882736156


The most appropriate distance is...
Cosine is the most appropriate distance and it does the best at classifying related documents. It helps overcome the flaw in 'count-the-common-words' used in Euclidean distance


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [43]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 

#cv = StratifiedShuffleSplit(ds.target, n_iter = 1, test_size = 0.5, train_size=0.5)

#cv = KFold(n_splits=2, random_state=323423, shuffle=False)

cv = StratifiedShuffleSplit(n_splits = 1, test_size = 0.5, train_size=0.5, random_state=10)



for trainidx, testidx in cv.split(ds.data, ds.target):
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K

for i in np.arange(1,30):
    clf = KNeighborsClassifier(n_neighbors=i, weights='uniform', metric='euclidean')
    clf.fit(X_train, y_train)
    curr_acc = accuracy_score(y_test, clf.predict(X_test))
    print(i, 'neighbor has acc of ', curr_acc)
    if acc < curr_acc:
        acc = curr_acc
        K = i
# overwrite this with the actual accuracy

#=====================================

print('Accuracy of classifier with %d neighbors is: %.2f'%(K,acc))


1 neighbor has acc of  0.5951917977726711
2 neighbor has acc of  0.5257203464733958
3 neighbor has acc of  0.5087502209651759
4 neighbor has acc of  0.48930528548700725
5 neighbor has acc of  0.48188085557716104
6 neighbor has acc of  0.47622414707442107
7 neighbor has acc of  0.46791585646102174
8 neighbor has acc of  0.45589535089269934
9 neighbor has acc of  0.4512992752342231
10 neighbor has acc of  0.44634965529432563
11 neighbor has acc of  0.44440516174650874
12 neighbor has acc of  0.43415237758529257
13 neighbor has acc of  0.4277885805197101
14 neighbor has acc of  0.42230864415768077
15 neighbor has acc of  0.4168287077956514
16 neighbor has acc of  0.40852041718225207
17 neighbor has acc of  0.40445465794590774
18 neighbor has acc of  0.40304048082022276
19 neighbor has acc of  0.400565670850274
20 neighbor has acc of  0.39667668375464027
21 neighbor has acc of  0.39261092451829593
22 neighbor has acc of  0.3874845324376878
23 neighbor has acc of  0.38624712745271345
24 nei

The best accuracy is 0.60 with 1 neighbors.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:


- Creating a tree with sparse data usually ends up being costly in comparison to just measuring the distance, so it does not make sense to create a KDTree. 
- Sparse data generates deep KDtrees because it has many zeros that cause it not to branch well.
___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [53]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
import warnings 
warnings.filterwarnings("ignore")



cv = StratifiedShuffleSplit(n_splits = 1, test_size = 0.5, train_size=0.5, random_state=10)


for trainidx, testidx in  cv.split(ds.data, ds.target):
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]


# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)
for d in ['l1','l2' ,'cosine']:
    clf = NearestCentroid(metric=d)
    clf.fit(X_train, y_train)
    y_hat = clf.predict(X_test)
    acc = accuracy_score(y_test, y_hat) 
    print('Accuracy for distance ', d, 'is', acc)

# fill in your code here


print('The best distance metric is: ', 'cosine')

Accuracy for distance  l1 is 0.3033409934594308
Accuracy for distance  l2 is 0.4111719992929114
Accuracy for distance  cosine is 0.4843556655471098
The best distance metric is:  cosine


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Enter you answer here:

*Double Click to edit*


In [26]:
# Use this space for any calculations you might want to do
conditionals = 130107*20
print('The total number of conditional probabilities that need to be parameterized is: ',conditionals)

print('20 priors need to be parameterized, one for each class')

The total number of conditional probabilities that need to be parameterized is:  2602140
20 priors need to be parameterized, one for each class


___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:

If the the intances contain mostly continous attributes it would be better to use a Gaussian distribution with the assumption that each class has its own Gaussian distribution.

For sparse data it would be to use a Multinomial Naive Bayse, which is a generalization of the Bernoulli Naive Bayes, which works best for binary data. 

___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
import warnings 
warnings.filterwarnings("ignore")







cv = StratifiedShuffleSplit(n_splits = 1, test_size = 0.5, train_size=0.5, random_state=10)


for trainidx, testidx in  cv.split(ds.data, ds.target):
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]


print('Varying alpha values')
for a in [0.0,0.001,0.01,1.0,2.0]:
    clf_mnb = MultinomialNB(alpha=a)
    clf_bnb = BernoulliNB(alpha=a, binarize=0.0)
    for i, clf in enumerate([clf_mnb,clf_bnb]):
            clf.fit(X_train,y_train)
            y_hat = clf.predict(X_test)
            acc = accuracy_score(y_test, y_hat) 
            print('Accuracy for model', i+1, 'with alpha', a, 'is', acc)

            
print('\n')   
print('Varying binarize values')            
for b in [0.0,0.001,0.01,1.0,2.0]:
    clf_bnb = BernoulliNB(alpha=1, binarize=b)
    clf.fit(X_train,y_train)
    y_hat = clf.predict(X_test)
    acc = accuracy_score(y_test, y_hat) 
    print('Binary model with binarize:', b, 'is', acc)

        
        

# fill in your code here

print('\n')            
print('These classifiers are so fast because...', 'Naive Bayes classifiers can be extremely fast compared to more sophisticated methods. The decoupling of the class conditional feature distributions help each distribution be independently estimated as a one dimensional distribution. This helps with the dimensionality reduction, thus reduce the calculation time.')
print('\n')     
print('The alpha values control...', 'Alpha is a smoothing parameter, using Naive Bayes there is the probability that a word is not in our dataset which could cause the zero probability. To fix this problem we can add an non-zero alpha, thus allowing for the probability to no longer be zero if a word is not present in our dataset. The higher the alpha the likelihood probability approaches a uniform distribution, (i.e the probability moves closer to 0.5) ')


Varying alpha values
Accuracy for model 1 with alpha 0.0 is 0.8566377938836839
Accuracy for model 2 with alpha 0.0 is 0.8458546933003359
Accuracy for model 1 with alpha 0.001 is 0.8872193742266219
Accuracy for model 2 with alpha 0.001 is 0.8322432384656179
Accuracy for model 1 with alpha 0.01 is 0.8817394378645925
Accuracy for model 2 with alpha 0.01 is 0.8156266572388191
Accuracy for model 1 with alpha 1.0 is 0.7180484355665547
Accuracy for model 2 with alpha 1.0 is 0.622945023864239
Accuracy for model 1 with alpha 2.0 is 0.6455718578751989
Accuracy for model 2 with alpha 2.0 is 0.5591302810677037


Varying binarize values
Binary model with binarize: 0.0 is 0.5591302810677037
Binary model with binarize: 0.001 is 0.5591302810677037
Binary model with binarize: 0.01 is 0.5591302810677037
Binary model with binarize: 1.0 is 0.5591302810677037
Binary model with binarize: 2.0 is 0.5591302810677037


These classifiers are so fast because... Naive Bayes classifiers can be extremely fast compar

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.